# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [23]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [8]:
features = spaceship.drop("Transported", axis=1)

features = features.select_dtypes(include=["number"])

target = spaceship["Transported"]

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

In [32]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [33]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

##### Gradient Boosting

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

gb_model = GradientBoostingClassifier(
    n_estimators=100,   # nombre d'arbres
    learning_rate=0.1,  # vitesse d'apprentissage
    max_depth=3,        # profondeur maximale de chaque arbre
    random_state=42
)

# train
gb_model.fit(X_train_norm, y_train)

# test prediction
y_pred_gb = gb_model.predict(X_test)

# evaluate
accuracy = accuracy_score(y_test, y_pred_gb)
print("Gradient Boosting Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred_gb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_gb))

Gradient Boosting Accuracy: 0.7347600518806745

Classification Report:
               precision    recall  f1-score   support

       False       0.72      0.76      0.74       762
        True       0.75      0.71      0.73       780

    accuracy                           0.73      1542
   macro avg       0.74      0.74      0.73      1542
weighted avg       0.74      0.73      0.73      1542

Confusion Matrix:
 [[578 184]
 [225 555]]


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [27]:
gradient_reg = GradientBoostingRegressor()

In [28]:
grid = {"n_estimators" : [300, 400, 500, 600, 750],
        "max_leaf_nodes": [15, 30, 50, 60, 75],
        "max_depth": [20, 30, 40]}

In [29]:
model = GridSearchCV(estimator = gradient_reg,
                     param_grid = grid,
                     cv = 5,
                     verbose = 10,
                     n_jobs=-1)

- Run Grid Search

In [36]:
model.fit(X_train_norm, y_train)

Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV 1/5; 1/75] START max_depth=20, max_leaf_nodes=15, n_estimators=300..........
[CV 3/5; 1/75] START max_depth=20, max_leaf_nodes=15, n_estimators=300..........
[CV 2/5; 1/75] START max_depth=20, max_leaf_nodes=15, n_estimators=300..........
[CV 5/5; 1/75] START max_depth=20, max_leaf_nodes=15, n_estimators=300..........
[CV 4/5; 1/75] START max_depth=20, max_leaf_nodes=15, n_estimators=300..........
[CV 3/5; 2/75] START max_depth=20, max_leaf_nodes=15, n_estimators=400..........
[CV 1/5; 2/75] START max_depth=20, max_leaf_nodes=15, n_estimators=400..........
[CV 4/5; 2/75] START max_depth=20, max_leaf_nodes=15, n_estimators=400..........
[CV 2/5; 2/75] START max_depth=20, max_leaf_nodes=15, n_estimators=400..........
[CV 5/5; 2/75] START max_depth=20, max_leaf_nodes=15, n_estimators=400..........
[CV 1/5; 1/75] END max_depth=20, max_leaf_nodes=15, n_estimators=300;, score=0.357 total time=   1.3s
[CV 3/5; 1/75] END max_dep

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'max_depth': [20, 30, 40],
                         'max_leaf_nodes': [15, 30, 50, 60, 75],
                         'n_estimators': [300, 400, 500, 600, 750]},
             verbose=10)

In [37]:
model.best_params_

{'max_depth': 20, 'max_leaf_nodes': 15, 'n_estimators': 300}

In [38]:
best_model = model.best_estimator_

In [39]:
best_model

GradientBoostingRegressor(max_depth=20, max_leaf_nodes=15, n_estimators=300)

- Evaluate your model

In [40]:
pred = best_model.predict(X_test_norm)


print("r2 score: ", r2_score(y_test, pred))
print("RMSE: ", root_mean_squared_error(y_test, pred))
print("MAE: ", mean_absolute_error(y_test, pred))

r2 score:  0.35046009981094783
RMSE:  0.40294273547140635
MAE:  0.3110910112061944


In [41]:
pred = best_model.predict(X_train_norm)


print("r2 score: ", r2_score(y_train, pred))
print("RMSE: ", root_mean_squared_error(y_train, pred))
print("MAE: ", mean_absolute_error(y_train, pred))

r2 score:  0.5951659046453088
RMSE:  0.3181303911889538
MAE:  0.2382901865098413
